# Automatização do processo de limpeza, processamento e importação de leads para o Salesforce

### O processo de automatização de leads levava horas para ficar pronto, mas com a automatização o processo foi reduzido para minutos. Abaixo segue o passo a passo, do código até a integração com o Salesforce.

In [1]:
import pandas as pd

file_path = r'C:\Users\camila.marques\Desktop\Camila\leads\leads.xlsx'
df = pd.read_excel(file_path)

df.head()

,Hora do envio: BRT,Nome completo,Enviar e-mail,Número de telefone
0,2024-06-04 20:58:00,Nicolas Hideto Okubo,okubonicolas@gmail.com,16993318970
1,2024-06-04 02:09:00,Helber Rafael Rodrigues de França,helberfranca.rr@gmail.com,+55 61991236159
2,2024-06-19 22:21:00,Tiago,ritakatiadealmeida@gmail.com,+55 66996981190
3,2024-06-03 14:04:00,WAGNER ALVES CHAGAS,wag2705@gmail.com,+55 51981546412
4,2024-06-17 08:26:00,ROBSON,npsd.rob.santos@gmail.com,+55 82994358552


In [2]:
df[['Primeiro Nome', 'Sobrenome']] = df['Nome completo'].str.split(n=1, expand=True)

df['Primeiro Nome'] = df['Primeiro Nome'].str.title()

df['Sobrenome'] = df.apply(lambda row: row['Primeiro Nome'][0].upper() if pd.isnull(row['Sobrenome']) else row['Sobrenome'].title(), axis=1)

In [3]:
df.rename(columns={'Enviar e-mail': 'Email'}, inplace=True)

df['Email'].head()

0          okubonicolas@gmail.com
1       helberfranca.rr@gmail.com
2    ritakatiadealmeida@gmail.com
3               wag2705@gmail.com
4       npsd.rob.santos@gmail.com
Name: Email, dtype: object

In [4]:
import re

def format_phone(phone):
    
    phone = re.sub(r'\D', '', phone)
    
    
    if phone.startswith('55'):
        phone = phone[2:]
    
    if len(phone) == 11:
        phone = f"({phone[:2]}){phone[2:]}"
    elif len(phone) == 10:
        phone = f"({phone[:2]}){phone[2:]}"
    
    return phone

df['Telefone'] = df['Número de telefone'].apply(format_phone)

df['Telefone'].head()

0    (16)993318970
1    (61)991236159
2    (66)996981190
3    (51)981546412
4    (82)994358552
Name: Telefone, dtype: object

In [5]:
num_rows_before = df.shape[0]

df.drop_duplicates(subset='Email', inplace=True)

num_rows_after = df.shape[0]

num_duplicates_removed = num_rows_before - num_rows_after

print(f"Número de duplicados removidos: {num_duplicates_removed}")

Número de duplicados removidos: 15


In [6]:
result_df = df[['Primeiro Nome', 'Sobrenome', 'Email', 'Telefone']]

result_df.to_excel(r'C:\Users\camila.marques\Desktop\Camila\leads\leads_Processado.xlsx', index=False)

In [ ]:
from simple_salesforce import Salesforce

sf = Salesforce(
    username='usuario',
    password='senha',
    security_token='token',
    domain='login'
)

# Testar a conexão
print(sf.describe())

# Preparar os dados para upload
records = result_df.to_dict(orient='records')

# Fazer upload dos dados para o Salesforce
for record in records:
    try:
        sf.Contact.create({
            'FirstName': record['Primeiro Nome'],
            'LastName': record['Sobrenome'],
            'Email': record['Email'],
            'Phone': record['Telefone']
        })
        print(f"Registro criado para {record['Email']}")
    except Exception as e:
        print(f"Erro ao criar registro para {record['Email']}: {e}")

Fim